In [9]:
from process_sql import tokenize

In [ ]:
#pip install -r requirements.txt

In [10]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\wasii\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping tokenizers\punkt.zip.


True

In [10]:
tokenize(r'SELECT * FROM AIRPORTS WHERE city  =  "Anthony"')

['select', '*', 'from', 'airports', 'where', 'city', '=', '"Anthony"']

In [14]:
import pickle

dataset = None

with open("dataset/classical_test.pkl", "rb") as file:
    dataset = pickle.load(file)

In [46]:
import datasets

In [48]:
import random

random.shuffle(dataset)

train_data = dataset[:int(len(dataset) * .8)]
test_data = dataset[int(len(dataset) * .8):]

In [69]:
inputs = [ex['text'] for ex in train_data]
targets = [ex['query'] for ex in train_data]

In [11]:
import sklearn
from sklearn.feature_extraction.text import CountVectorizer

In [65]:
import transformers
from transformers import PreTrainedTokenizerFast
from tokenizers import Tokenizer
from tokenizers.models import BPE
from tokenizers.trainers import BpeTrainer
from tokenizers.pre_tokenizers import Whitespace
from tokenizers import Tokenizer

In [66]:
source_tokenizer = Tokenizer(BPE(unk_token="[UNK]"))
source_tokenizer_trainer = BpeTrainer(special_tokens=["[UNK]", "[PAD]", "[BOS]", "[EOS]"])
source_tokenizer.pre_tokenizer = Whitespace()

source_iterator = (item for item in train_data)
source_tokenizer.train_from_iterator(
    source_iterator,
    trainer=source_tokenizer_trainer,
)

source_tokenizer = transformers.PreTrainedTokenizerFast(
    tokenizer_object=source_tokenizer,
    bos_token="[BOS]",
    eos_token="[EOS]",
    pad_token="[PAD]",
)

In [ ]:
source_tokenizer(inputs, max_length=100, truncation=True)

In [55]:
target_vectorizer = CountVectorizer(lowercase=True, tokenizer=tokenize)

In [59]:
target_vectorizer = target_vectorizer.fit(targets)

C:\Users\wasii\anaconda3\envs\NLP_class\lib\site-packages\sklearn\feature_extraction\text.py:517: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  "The parameter 'token_pattern' will not be used"


In [61]:
targets_vectorized = target_vectorizer.transform(targets)

In [74]:
target_vectorizer.transform(['return me the papers on PVLDB']).toarray()

array([[0, 0, 0, ..., 0, 0, 0]], dtype=int64)

In [62]:
target_vectorizer.vocabulary_

{'select': 965,
 'count': 547,
 '(': 281,
 'distinct': 656,
 'course_offeringalias0.course_id': 551,
 ')': 282,
 'from': 761,
 'course_offering': 549,
 'as': 437,
 'course_offeringalias0': 550,
 ',': 285,
 'semester': 966,
 'semesteralias0': 967,
 'where': 1026,
 'semesteralias0.semester': 968,
 '=': 383,
 '"fa"': 91,
 'and': 432,
 'semesteralias0.semester_id': 969,
 'course_offeringalias0.semester': 558,
 'semesteralias0.year': 970,
 '2016': 329,
 ';': 380,
 'coursealias0.department': 576,
 'coursealias0.name': 582,
 'coursealias0.number': 583,
 'course': 548,
 'coursealias0': 572,
 'inner': 793,
 'join': 802,
 'on': 867,
 'coursealias0.course_id': 574,
 'program_course': 898,
 'program_coursealias0': 899,
 'program_coursealias0.course_id': 901,
 '"eecs"': 86,
 'program_coursealias0.category': 900,
 'like': 822,
 '"%ulcs%"': 12,
 'in': 792,
 '"sp"': 233,
 '"ss"': 234,
 '"su"': 240,
 'coursealias0.description': 577,
 '"%artificial intelligence%"': 0,
 'or': 868,
 'coursealias0.credits'

In [76]:
len(target_vectorizer.vocabulary_)

1054

In [44]:
word_id_tups = list(target_vectorizer.vocabulary_.items())

word_id_tups

[('select', 1004),
 ('journalalias0.homepage', 843),
 ('from', 798),
 ('journal', 841),
 ('as', 466),
 ('journalalias0', 842),
 ('where', 1065),
 ('journalalias0.name', 845),
 ('=', 408),
 ('"pvldb"', 220),
 (';', 405),
 ('authoralias0.homepage', 473),
 ('author', 468),
 ('authoralias0', 469),
 ('authoralias0.name', 474),
 ('"h. v. jagadish"', 114),
 ('publicationalias0.abstract', 955),
 ('publication', 949),
 ('publicationalias0', 954),
 ('publicationalias0.title', 961),
 ('"making database systems usable"', 156),
 ('publicationalias0.year', 962),
 ('>', 409),
 ('2000', 342),
 ('conferencealias0.homepage', 572),
 ('conference', 569),
 ('conferencealias0', 570),
 ('conferencealias0.name', 573),
 ('"vldb"', 290),
 ('keywordalias0.keyword', 853),
 ('keyword', 850),
 ('keywordalias0', 851),
 ('organizationalias0.name', 912),
 ('organization', 908),
 ('organizationalias0', 909),
 ('organizationalias0.continent', 910),
 ('"north america"', 196),
 ('organizationalias0.homepage', 911),
 ('"un

In [41]:
BOS_INDEX = 0
EOS_INDEX = len(target_vectorizer.vocabulary_) + 1
PAD_INDEX = len(target_vectorizer.vocabulary_) + 2

1095

In [80]:
import numpy as np

(target_vectorizer.transform(targets) > 0).nonzero()

(array([   0,    0,    0, ..., 2806, 2806, 2806]),
 array([  91,  281,  282, ...,  994,  996, 1026]))

In [88]:
def build_ids_vector(targets):
    arr = []
    for row in target_vectorizer.transform(targets).toarray():
        arr.append([0] + (row > 0).nonzero()[0])

    return arr

In [89]:
build_ids_vector(targets)

[array([  91,  281,  282,  285,  329,  380,  383,  432,  437,  547,  549,
         550,  551,  558,  656,  761,  965,  966,  967,  968,  969,  970,
        1026], dtype=int64),
 array([  12,   86,  233,  234,  240,  281,  282,  285,  329,  380,  383,
         432,  437,  548,  549,  550,  551,  558,  572,  574,  576,  582,
         583,  656,  761,  792,  793,  802,  822,  867,  898,  899,  900,
         901,  965,  966,  967,  968,  969,  970, 1026], dtype=int64),
 array([   0,  281,  282,  285,  350,  380,  383,  432,  437,  548,  572,
         575,  576,  577,  582,  583,  656,  761,  822,  868,  965, 1026],
       dtype=int64),
 array([ 122,  272,  285,  380,  383,  386,  387,  388,  392,  394,  395,
         396,  432,  437,  472,  473,  474,  475,  477,  478,  479,  761,
         845,  846,  848,  850,  965, 1026], dtype=int64),
 array([   6,  270,  330,  380,  383,  432,  437,  548,  549,  550,  551,
         555,  558,  572,  574,  656,  761,  793,  794,  795,  796,  797,
     